In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from tqdm import tqdm
import cv2

def extract_color_histogram(image, bins=8):
    """
    Extrahiert einen normalisierten Farbhistogramm-Vektor aus einem Bild (RGB, uint8).
    """
    chans = cv2.split(image)
    features = []
    for chan in chans:
        hist = cv2.calcHist([chan], [0], None, [bins], [0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        features.extend(hist)
    return np.array(features)

def run_cifar10_experiment_batchwise(bins=8):
    # CIFAR-10 mit transform zum Tensor laden
    transform = transforms.Compose([transforms.ToTensor()])
    cifar10_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform
    )
    
    # DataLoader: Batchweise Laden, damit nicht alle Bilder gleichzeitig im Speicher sind
    loader = torch.utils.data.DataLoader(cifar10_dataset, batch_size=64, shuffle=False)
    
    all_features = []
    all_labels = []
    
    # Bilder batchweise verarbeiten
    for images, labels in tqdm(loader, desc="Batches verarbeiten"):
        # images: Tensor der Form (B, C, H, W)
        for img, label in zip(images, labels):
            # Von Tensor (C, H, W) zu numpy (H, W, C) konvertieren
            img_np = img.permute(1, 2, 0).numpy()
            # Umwandeln in uint8
            img_np = (img_np * 255).astype(np.uint8)
            feat = extract_color_histogram(img_np, bins=bins)
            all_features.append(feat)
            all_labels.append(label.item())
    
    all_features = np.array(all_features)
    all_labels = np.array(all_labels)
    print("Extraktion abgeschlossen. Features-Shape:", all_features.shape)
    
    # Hier kannst du dann mit den extrahierten Features weitermachen (z.B. Train-Test-Split, Klassifikator-Training usw.)
    # ...
    
    return all_features, all_labels

if __name__ == "__main__":
    features, labels = run_cifar10_experiment_batchwise(bins=8)


100%|██████████| 170M/170M [00:09<00:00, 18.1MB/s] 
Batches verarbeiten: 100%|██████████| 782/782 [00:02<00:00, 270.96it/s]

Extraktion abgeschlossen. Features-Shape: (50000, 24)
